# 01_download (housing)
**Authors**: <fill names> | **Owner**: <name> | **Reviewer**: <name> | **Date**: 2025-11-07

**Purpose:** Download Kaggle UK housing data (England/Wales), standardize columns, and write partitioned Parquet.

> Tip: Keep code blocks small. Explain choices (NA strategy, outlier handling, feature decisions) in Markdown right above the code that implements them.


In [10]:
!pip install pandas kaggle --quiet
!pip install pyarrow fastparquet --quiet



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
import sys
!{sys.executable} -m pip install pyarrow fastparquet



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



### Setup
- Requires Kaggle API credentials (`~/.kaggle/kaggle.json`).
- Adjust `KAGGLE_DATASET` and file names to match the dataset you're using.
- This notebook **does not** commit data to git; it writes to `data/raw/housing/` and `data/intermediate/`.


In [12]:
from pathlib import Path
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Folder where your CSV is stored
RAW_DIR = Path("price_paid_records")
INT_DIR = Path("intermediate_data")

# Create directories if they don't exist
RAW_DIR.mkdir(parents=True, exist_ok=True)
INT_DIR.mkdir(parents=True, exist_ok=True)

print("Directories ready:", RAW_DIR, INT_DIR)

# Now point to your actual CSV file
CSV_PATH = RAW_DIR / "price_paid_records.csv"

# Check if it exists
assert CSV_PATH.exists(), f"File not found: {CSV_PATH}"
print("Using file:", CSV_PATH)


Directories ready: price_paid_records intermediate_data
Using file: price_paid_records\price_paid_records.csv


### (Option A) Download with Kaggle API

In [13]:

# KAGGLE_DATASET = "INSERT/SLUG"   # e.g., "hm-land-registry/uk-housing-prices-paid"
# !kaggle datasets download -d $KAGGLE_DATASET -p $RAW_DIR --unzip


### (Option B) Proceed from an existing CSV

In [14]:
# Point to your CSV file (2GB+). If there are multiple files, loop over them.

# If you already have a cleaned file somewhere else, you can point directly to it here:

# Try to locate a CSV in the raw folder first
csv_files = list(RAW_DIR.glob("*.csv"))
if csv_files:
    CSV_PATH = csv_files[0]
    print(f"Using raw CSV: {CSV_PATH}")
else:
    raise FileNotFoundError(
        "No CSV file found in ../data/raw/housing or ../cleanedDataStored. "
        "Please add your dataset first."
    )

CSV_PATH


Using raw CSV: price_paid_records\price_paid_records.csv


WindowsPath('price_paid_records/price_paid_records.csv')

### Convert to partitioned Parquet (by year, region)

In [15]:
def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.str.strip()
                  .str.lower()
                  .str.replace(r"\s+", "_", regex=True)
                  .str.replace(r"[^0-9a-zA-Z_]+", "", regex=True)
    )
    return df

def prepare_chunk(df: pd.DataFrame) -> pd.DataFrame:
    df = normalize_columns(df)

    # Parse a date column if present (typical PPD column is 'date_of_transfer')
    date_cols = [c for c in df.columns if "date" in c or "transfer" in c]
    if date_cols:
        dc = date_cols[0]
        df[dc] = pd.to_datetime(df[dc], errors="coerce")
        df["year"] = df[dc].dt.year
        df["month"] = df[dc].dt.month
    else:
        df["year"] = pd.NA
        df["month"] = pd.NA

    # Try to build a 'region' column if you have county/district/etc.
    region_cols = [c for c in ["county","district","local_authority","region","town_city"] if c in df.columns]
    if region_cols:
        df["region"] = df[region_cols[0]].astype("string")
    else:
        df["region"] = pd.Series(pd.Categorical(["unknown"] * len(df))).astype("string")

    # Ensure price is numeric if present
    if "price" in df.columns:
        df["price"] = pd.to_numeric(df["price"], errors="coerce")

    return df



In [17]:
out_csv = INT_DIR / "housing_all.csv"
header_written = out_csv.exists() and out_csv.stat().st_size > 0
row_count = 0

for i, chunk in enumerate(pd.read_csv(CSV_PATH, chunksize=1_000_000, low_memory=False), 1):
    chunk = prepare_chunk(chunk)
    # Append to CSV; write header only once
    chunk.to_csv(out_csv, mode="a", index=False, header=not header_written)
    header_written = True
    row_count += len(chunk)
    if i % 5 == 0:
        print(f"Written ~{row_count:,} rows to {out_csv}")

print(f"Wrote: {out_csv} (rows: ~{row_count:,})")

# ================================================
# 2) OPTIONAL: WRITE PARTITIONED CSVs BY YEAR
# ================================================
part_dir = INT_DIR / "partitioned_csv"
part_dir.mkdir(parents=True, exist_ok=True)

# Track which year files we've already created (to control headers)
headers_written = set()

for chunk in pd.read_csv(CSV_PATH, chunksize=1_000_000, low_memory=False):
    chunk = prepare_chunk(chunk)
    if "year" not in chunk.columns:
        continue

    # Drop rows without a year
    chunk = chunk.dropna(subset=["year"])
    if chunk.empty:
        continue

    chunk["year"] = chunk["year"].astype("Int64")

    for yr, g in chunk.groupby("year", sort=False):
        out_path = part_dir / f"year={int(yr)}.csv"
        write_header = int(yr) not in headers_written and not out_path.exists()
        g.to_csv(out_path, mode="a", index=False, header=write_header)
        headers_written.add(int(yr))

print(f"Partitioned CSV files written to: {part_dir}")

Written ~5,000,000 rows to intermediate_data\housing_all.csv
Written ~10,000,000 rows to intermediate_data\housing_all.csv
Written ~15,000,000 rows to intermediate_data\housing_all.csv
Written ~20,000,000 rows to intermediate_data\housing_all.csv
Wrote: intermediate_data\housing_all.csv (rows: ~21,550,210)
Partitioned CSV files written to: intermediate_data\partitioned_csv
